In [11]:
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

ROOT = Path.cwd()
stocks_file = ROOT / 'stocks.txt'
tickers = [line.strip().upper() for line in stocks_file.read_text().splitlines() if line.strip()]
tickers

['TSLA',
 'NVDA',
 'MSFT',
 'META',
 'PLTR',
 'MSTR',
 'ASML',
 'AMD',
 'AVGO',
 'ALAB',
 'MRVL',
 'BTC-USD']

In [12]:
from script import analyze_ticker

def run_batch(ticker_list, concurrency=6):
    results = []
    with ThreadPoolExecutor(max_workers=concurrency) as ex:
        futures = {ex.submit(analyze_ticker, t): t for t in ticker_list}
        for fut in as_completed(futures):
            results.append(fut.result())
    return pd.DataFrame(results)

df = run_batch(tickers)
df = df.round(2)  # Round all numeric values to 2 decimal places
df

,ticker,signal,current_price,price_note,date,d20,d50,d100,d200,w10,...,weekly_poc,weekly_vah,weekly_val,s1,s2,s3,r1,r2,r3,notes
0,TSLA,FULL HOLD + ADD,438.07,last close,2026-01-03,464.27,445.01,418.45,360.20,442.77,...,330.93,396.38,219.94,411.45,382.78,373.04,488.54,474.07,470.75,
1,PLTR,FULL HOLD + ADD,167.86,last close,2026-01-03,184.55,180.94,176.47,151.04,179.10,...,87.90,161.40,69.16,147.56,142.34,128.51,207.52,190.00,188.20,
2,MSTR,CASH,157.16,last close,2026-01-03,167.75,202.40,267.75,320.20,179.39,...,318.86,389.58,235.84,155.61,120.23,113.69,543.00,457.22,430.35,
3,META,HOLD MOST → REDUCE,650.41,last close,2026-01-03,658.59,651.82,698.45,672.61,640.47,...,709.65,750.78,586.28,580.78,578.18,546.88,795.06,789.62,758.54,
4,MSFT,HOLD MOST → REDUCE,472.94,last close,2026-01-03,483.17,495.21,503.04,476.73,485.44,...,509.29,553.50,410.33,464.89,407.71,404.37,553.50,552.69,530.04,
5,NVDA,FULL HOLD + ADD,188.85,last close,2026-01-03,182.96,186.50,183.02,160.48,183.82,...,115.49,177.02,86.60,176.75,169.54,164.05,212.18,NaN,NaN,
6,AMD,HOLD,223.47,last close,2026-01-03,214.41,227.63,203.23,163.23,219.71,...,162.25,251.83,129.85,194.28,161.81,153.34,267.08,227.30,NaN,
7,ASML,FULL HOLD + ADD,1163.78,last close,2026-01-03,1083.21,1056.14,965.47,843.33,1063.95,...,735.85,945.33,574.25,988.40,946.11,933.75,NaN,NaN,NaN,
8,ALAB,HOLD MOST → REDUCE,179.56,last close,2026-01-03,162.40,160.76,179.74,134.57,161.72,...,165.80,245.64,107.54,148.51,131.42,84.78,262.90,201.86,199.47,
9,AVGO,HOLD,347.62,last close,2026-01-03,360.65,361.22,344.91,288.92,359.56,...,344.68,413.82,220.24,328.43,323.43,280.85,413.82,385.74,372.88,


In [13]:
# Export results to CSV with 2 decimal places
import shutil
from pathlib import Path

output_file = ROOT / 'batch_results.csv'
df_rounded = df.round(2)
df_rounded.to_csv(output_file, index=False)
print(f"Results exported to: {output_file}")

# Copy to Downloads folder
downloads_path = Path.home() / 'Downloads' / 'batch_results.csv'
shutil.copy(output_file, downloads_path)
print(f"File also copied to: {downloads_path}")

# Optional: Export to Excel (requires openpyxl)
# df_rounded.to_excel(ROOT / 'batch_results.xlsx', index=False)

Results exported to: c:\workspace\my_script_project\batch_results.csv
File also copied to: C:\Users\karms\Downloads\batch_results.csv
